# Applied

__5.__ In Chapter 4, we used logistic regression to predict the probability of `default` using `income` and `balance` on the `Default` data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not foget to set a random seed before beginning your analysis.

(a) Fit a logistic regression model that uses `income` and `balance` to predict `default`

In [1]:
library(ISLR2)
attach(Default)

In [2]:
set.seed(1)
fit.glm <- glm(default ~ income + balance, data = Default, family = binomial)
summary(fit.glm)


Call:
glm(formula = default ~ income + balance, family = binomial, 
    data = Default)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

In [3]:
train <- sample(dim(Default)[1], dim(Default)[1] / 2)

ii. Fit a multiple logistic regression model using only the training observations.

In [4]:
fit.glm <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train)
summary(fit.glm)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default, subset = train)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.194e+01  6.178e-01 -19.333  < 2e-16 ***
income       3.262e-05  7.024e-06   4.644 3.41e-06 ***
balance      5.689e-03  3.158e-04  18.014  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1523.8  on 4999  degrees of freedom
Residual deviance:  803.3  on 4997  degrees of freedom
AIC: 809.3

Number of Fisher Scoring iterations: 8


iii. Obtain a prediction of default status for each individual in default for that individual, and classifying the individual to the `default` category if the posterior probability is greater than $0.5$.

In [5]:
pred <- predict(fit.glm, newdata = Default[-train,], type = "response")
pred.glm <- rep("No", length(pred))
pred.glm[pred > 0.5] <- "Yes"

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [6]:
vse <- c(mean(pred.glm != Default[-train, ]$default))
vse

[1] 0.0254

We have a $2.54\%$ test error rate with the vaildation set approach.

(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [7]:
for (i in 1:3) {
    train <- sample(dim(Default)[1], dim(Default)[1] / 2)
    fit.glm <- glm(default ~ income + balance, data = Default, family = "binomial", subset = train)
    pred <- predict(fit.glm, newdata = Default[-train,], type = "response")
    pred.glm <- rep("No", length(pred))
    pred.glm[pred > 0.5] <- "Yes"
    vse <- c(vse, mean(pred.glm != Default[-train,]$default))
}
for (i in 1:length(vse)) {
    cat("Test Error Rate for attempt", i, ":", vse[i]*100, "%", "\n")
}

Test Error Rate for attempt 1 : 2.54 % 
Test Error Rate for attempt 2 : 2.74 % 
Test Error Rate for attempt 3 : 2.44 % 
Test Error Rate for attempt 4 : 2.44 % 


We see that the validation estimate of the test error rate can be variable, depending on precisely which observations are included in the training set and which observations are in the validation set.

(d) Now consider a logistic regression model that predicts the probability of `default` using `income`, `balance`, and a dummy variable for `student`. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for `student` leads to a reduction in the test error rate.

In [8]:
train <- sample(dim(Default)[1], dim(Default)[1] / 2)
fit.glm <- glm(default ~ income + balance + student, data = Default, family = "binomial", subset = train)
pred.glm <- rep("No", length(pred))
pred <- predict(fit.glm, newdata = Default[-train,], type = "response")
pred.glm[pred > 0.5] <- "Yes"
mean(pred.glm != Default[-train,]$default)

[1] 0.0278

Adding the `student` dummy variable does not result in a reduction in the test error rate.

---

__6.__ We continue to consider the use of a logistic regression model to predict the probability of `default` using `income` and `balance` on the `Default` data set. In particular, we will now compute estimates for the standard errors of the `income` and `balance` logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the `glm()` function. Do not forget to set a random seed before beginning your analysis.

(a) Using the `summary()` and `glm()` functions, determine the estimated standard errors for the coefficients associated with `income` and `balance` in a multiple logistic regression model that uses both predictors.

We already utilized this in 5(a).

In [9]:
set.seed(1)
fit.glm <- glm(default ~ income + balance, data = Default, family = binomial)
(fit.sum <- summary(fit.glm))


Call:
glm(formula = default ~ income + balance, family = binomial, 
    data = Default)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


In [10]:
cat("Estimated Standard errors for the model:\n")
coef(fit.sum)[,2]

Estimated Standard errors for the model:


(Intercept)       income      balance 
4.347564e-01 4.985167e-06 2.273731e-04

(b) Write a function, `boot.fn()`, that takes as input the `Default` data set as well as an index of the observations, and that outputs the coefficient estmates for `income` and `balance` in the multiple logistic model.

In [11]:
boot.fn <- function(data, index) {
    fit <- glm(default ~ income + balance, data = data, family = binomial, subset = index)
    return(coef(fit))
}

(c) Use the `boot()` function together with your `boot.fn()` function to estimate the standard errors of the logistic regression coefficients for `income` and `balance`.

In [12]:
library(boot)
boot(Default, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Default, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
         original        bias     std. error
t1* -1.154047e+01 -3.945460e-02 4.344722e-01
t2*  2.080898e-05  1.680317e-07 4.866284e-06
t3*  5.647103e-03  1.855765e-05 2.298949e-04

The bootstrap estimates of the standard errors for the coefficients  $\beta_0$, $\beta_1$, and $\beta_2$.

(d) Comment on the estimated standard errors obtained using the `glm()` function and using your bootstrap function.

The estimated standard errors between the two is pretty close.